In [1]:
# J'importe les modules nécessaires pour le script.
import requests  # Pour effectuer des requêtes HTTP.
from bs4 import BeautifulSoup  # Pour parser le contenu HTML.
import pandas as pd  # Pour manipuler et stocker les données sous forme de tableaux.
import os  # Pour interagir avec le système de fichiers de l'ordinateur.
import time  # Pour introduire des délais dans l'exécution du script.
import random  # Pour générer des nombres aléatoires.

# Je définis les en-têtes HTTP pour simuler une requête d'un navigateur web moderne.
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Je spécifie le chemin du dossier où les images téléchargées seront enregistrées.
img_folder_path = 'D:/Images_champignons'
# Je vérifie si le dossier spécifié existe et sinon, je le crée.
if not os.path.exists(img_folder_path):
    os.makedirs(img_folder_path)

# J'initialise une session de requêtes HTTP pour maintenir les cookies et autres en-têtes à travers les requêtes.
session = requests.Session()
session.headers.update(headers)  # J'ajoute les en-têtes définis à la session.

# Je fixe l'URL de base du site que je souhaite scraper.
base_url = 'https://mushroomobserver.org/'

# Je prépare une liste pour stocker les données extraites.
data = []

# Je boucle sur les premières trois pages du site pour l'exemple.
for i in range(1, 4):
    # Je construis l'URL de chaque page en combinant l'URL de base avec le numéro de la page.
    page_url = f'{base_url}{i}'
    # Je récupère un identifiant unique pour chaque page, ici simplement le numéro de la page.
    url_reference = str(i)
    # Je crée une liste pour stocker les noms des images trouvées sur la page.
    img_names = []
    
    # J'introduis un délai aléatoire entre 1 et 5 secondes avant de faire la requête pour simuler un comportement humain.
    sleep_time = random.randint(1, 5)
    time.sleep(sleep_time)
    
    # J'essaie d'effectuer une requête HTTP pour obtenir le contenu de la page.
    try:
        response = session.get(page_url)
        response.raise_for_status()  # Je vérifie que la requête a réussi.
        soup = BeautifulSoup(response.content, 'html.parser')  # Je parse le contenu HTML obtenu.
        
        # Je cherche le nom du champignon dans la balise <h1> avec l'ID 'title'.
        name_tag = soup.find('h1', id='title').find('i')
        name = name_tag.get_text(strip=True) if name_tag else f'Unknown_{i}'  # Si la balise <i> n'existe pas, j'utilise un nom par défaut.
        
        # Je parcours toutes les balises <img> avec la classe 'carousel-image' pour extraire les URLs des images.
        img_tags = soup.find_all('img', class_='carousel-image')
        img_counter = 1
        for img_tag in img_tags:
            img_url = img_tag.get('data-src')  # J'extrais l'URL de l'image.
            if img_url:
                img_data = session.get(img_url).content  # Je télécharge l'image.
                # Je crée un nom de fichier unique pour chaque image.
                img_name = f"{url_reference}_{name.replace(' ', '_')}_{img_counter}.jpg"
                img_path = os.path.join(img_folder_path, img_name)  # Je construis le chemin du fichier.
                
                # J'ouvre un fichier pour écrire les données de l'image.
                with open(img_path, 'wb') as f:
                    f.write(img_data)
                
                img_names.append(img_name)  # J'ajoute le nom de l'image à la liste.
                img_counter += 1  # J'incrémente le compteur d'image.
        
        # Je crée un dictionnaire pour stocker les données de cette page, y compris l'URL, le nom, et les noms d'images.
        page_data = {'URL': page_url, 'Name': name}
        for j, img_name in enumerate(img_names, start=1):
            page_data[f'image_{j}'] = img_name
        
        data.append(page_data)  # J'ajoute les données de la page à la liste principale.
    
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la requête {page_url} : {e}")  # J'affiche les erreurs rencontrées pendant la requête.
        continue

# Je convertis la liste des données collectées en un DataFrame pour une manipulation facile.
df = pd.DataFrame(data)
# Je redéfinis les colonnes du DataFrame pour inclure toutes les images trouvées.
max_images = max(len(d) for d in data) - 2
df = df.reindex(columns=['URL', 'Name'] + [f'image_{j}' for j in range(1, max_images + 1)])

# J'affiche le DataFrame pour vérifier les données extraites.
print(df)

# Je sauvegarde le DataFrame au format Excel pour une utilisation ultérieure.
output_path = 'D:/Images_champignons/mushroom_data.xlsx'
df.to_excel(output_path, index=False)

# Je confirme que les données ont été sauvegardées avec succès.
print(f'Données sauvegardées à {output_path}')


                              URL                Name  \
0  https://mushroomobserver.org/1  Xylaria polymorpha   
1  https://mushroomobserver.org/2   Xylaria magnoliae   
2  https://mushroomobserver.org/3   Xylaria hypoxylon   

                      image_1  
0  1_Xylaria_polymorpha_1.jpg  
1   2_Xylaria_magnoliae_1.jpg  
2   3_Xylaria_hypoxylon_1.jpg  
Données sauvegardées à D:/Images_champignons/mushroom_data.xlsx
